Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE        MCSE      ESS
alpha 154.5987095 0.269752157 0.0042651561 0.00484862396 1000
 beta   0.9056246 0.043434303 0.0006867566 0.00054467685 1000
sigma   5.1083906 0.194572987 0.0030764691 0.00308840791 1000

Quantiles:
          2.5%       25.0%       50.0%     75.0%      97.5%   
alpha 154.0680000 154.4210000 154.602000 154.77300 155.1270500
 beta   0.8190580   0.8766512   0.905499   0.93483   0.9909783
sigma   4.7431082   4.9739225   5.101720   5.23753   5.5032607



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.609 0.946325 5.06585; 154.618 0.885629 5.04991; … ; 154.863 0.912442 5.14777; 154.681 0.982042 5.32227]

Union{Missing, Float64}[154.332 0.927913 5.487; 154.97 0.875191 4.80315; … ; 154.1 0.822433 5.32499; 154.239 0.869424 5.31903]

Union{Missing, Float64}[154.801 0.904086 4.78935; 154.733 0.942529 5.08136; … ; 154.784 0.936018 4.79337; 154.672 0.879925 4.98819]

Union{Missing, Float64}[154.429 0.872919 5.26986; 154.484 0.931671 4.90187; … ; 154.643 0.973413 4.86035; 154.483 0.857256 5.08221]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE        MCSE      ESS
alpha 154.5987095 0.269752157 0.0042651561 0.00484862396 1000
 beta   0.9056246 0.043434303 0.0006867566 0.00054467685 1000
sigma   5.1083906 0.194572987 0.0030764691 0.00308840791 1000

Quantiles:
          2.5%       25.0%       50.0%     75.0%      97.5%   
alpha 154.0680000 154.4210000 154.602000 154.77300 155.1270500
 beta   0.8190580   0.8766512   0.905499   0.93483   0.9909783
sigma   4.7431082   4.9739225   5.101720   5.23753   5.5032607



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*